<a align="center">
    <img src="FairyTail.webp" width="500">
</a>

<font color='green' size= 6 ><b>Install the following library if not already installed</b></font>

##### Pandas: %pip install pandas

##### Python Dotenv: %pip install python-dotenv

##### Requests: %pip install requests

##### Beautiful soup: %pip install bs4

##### MySQL Connector: %pip install mysql-connector-python

##### Selenium: %pip install selenium

<font color='green' size= 6 ><b>Import Library necessary for the Project and define the variables and functions</b></font>

In [7]:
# importing os ans sys modules
import os
import sys

# Navigate to project directory to import modules

sys.path.insert(0, os.path.dirname(os.getcwd()))

# Importing function create from Manga_function.py
from Functions.Manga_function import write_to_file, create_soup, extract_date, extract_episod, extract_link, extract_title 

# MySQL connector
import mysql.connector

# Pandas
import pandas as pd

# importing os module for environment variables
import os

# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv() 


True

In [8]:
# Define the variables

host = os.getenv("MYSQL_HOSTNAME")
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
database = "manga"
table_name = "FairyTail"
manga_name = "Fairy Tail"
csv_file = "C:/Users/yacbo/Documents/Data_Engineer/Training/Manga_Project/FairyTail/FairyTail.csv"
log_file = "C:/Users/yacbo/Documents/Data_Engineer/Training/Manga_Project/FairyTail/FairyTail.txt"
url = os.getenv("URL")
listofseason = [1,2,3,4,5,6,7,8]
nb_ep_season = [48,48,50,25,50,39,15,50]
number_tot_ep = sum(nb_ep_season)

In [9]:
# function to extract and transform date 

def extract_date(date):
    """Extract date from the beautiful soup..

    Args:
        date (str): String including the date extracted from beautiful soup.
    """

    date = date.split(",")[1].strip()

    day = int(date.split(" ")[0])

    month = int(date.split(" ")[1].replace("janv.","01").replace("févr.","02").replace("mars","03").replace("avr.","04").replace("mai","05").replace("juin","06").replace("juil.","07").replace("août","08").replace("sept.","09").replace("oct.","10").replace("nov.","11").replace("déc.","12"))

    year = int(date.split(" ")[2])

    return day,month,year

<font color='green' size= 6 ><b>Run "Manga_init.ipynb"</b></font>

In [10]:
%run C:\Users\yacbo\Documents\Data_Engineer\Training\Manga_Project\Manga_init.ipynb

In [11]:
if number_ep_to_extract == 0 :
    write_to_file(log_file, f"Program stopped because there is no data to extract")
    raise SystemExit("System stop because there is no episod to extract")

<font color='green' size= 6 ><b>ETL Batching </b></font>

In [12]:
# Run "FairyTail_ETL.ipynb" while there are some episods to extract
while number_ep_to_extract > 0:   
    nb = 0
    for i in range(len(listofseason)):
        nb = nb + nb_ep_season[i]
        if nb > number_ep_database:
            season_number = listofseason[i]
            number_ep_season_to_extract = nb - number_ep_database
            break

    %run C:\Users\yacbo\Documents\Data_Engineer\Training\Manga_Project\FairyTail\FairyTail_ETL.ipynb

    # Extract number of episod in database
    try:
            # Execute a query
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        number_ep_database = cursor.fetchall()
        number_ep_database = int(number_ep_database[0][0])
        write_to_file(log_file, f"There is(are) {number_ep_database} episod(s) already in {table_name} table")

    except mysql.connector.Error as err:
        write_to_file(log_file, f"There is an error : {err}")
        # Close the database cursor and connection
        if cursor:
            cursor.close()
        if cnx:
            cnx.close()
        raise

    number_ep_to_extract = number_tot_ep - number_ep_database
    write_to_file(log_file, f"There is(are) {number_ep_to_extract} episod(s) to extract")

# Close the database cursor and connection
if cursor:
    cursor.close()
if cnx:
    cnx.close()